# Extracting location names and data from Gibbon

In this note book we will put together and practice a lot of skills we have learned so far this term. Starting with just the raw text files from Gibbon's Decline and Fall we will create a DataFrame containing location names, location counts, and location data.

The code in this notebook may seem complex, but if you read through it carefully, you will likely understand what most of the code is doing.


## Set-up

In [ ]:
# install necessary libraries. The "%%capture" stops the notebook from printing
# out all the install output. Remove if you need to trouble shoot.
!pip install stanza

In [ ]:
# install necessary libraries. The "%%capture" stops the notebook from printing
# out all the insall output. Remove if you need to trouble shoot.

!pip install wget

In [3]:
# import necessary libraries
import os
import pandas as pd
import stanza
import json
import wget



## NLP pipeline
Now that all the necessary libraries have been installed and imported into our project, we need to set up our nlp pipeline. We will use [Stanza](https://stanfordnlp.github.io/stanza/).

In [4]:
# load stanza nlp pipeline that tokenizes and performs Named Entity Recognition
nlp_ner= stanza.Pipeline(lang='en', processors='tokenize, ner')

2023-11-03 14:27:38 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-03 14:27:39 INFO: Loading these models for language: en (English):
| Processor | Package          |
--------------------------------
| tokenize  | combined         |
| ner       | ontonotes_charlm |

2023-11-03 14:27:39 INFO: Using device: cpu
2023-11-03 14:27:39 INFO: Loading: tokenize
2023-11-03 14:27:39 INFO: Loading: ner
2023-11-03 14:27:40 INFO: Done loading processors!


## Load text data
If you are using a **Colab Notebook** you will need to run the cell below to get the text files.

Otherwise, you should have all of the text files for Gibbon's _Decline and Fall of the Roman Empire_ already downloaded from Canvas.

In [5]:
# load text files, Colab only
! git clone https://github.com/jdeen33/Gibbon_text.git

Cloning into 'Gibbon_text'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 74 (delta 0), reused 0 (delta 0), pack-reused 71
Receiving objects: 100% (74/74), 4.20 MiB | 7.71 MiB/s, done.


## Extract location infromation from text file(s)

In [6]:
# create function that will take a text string as input and return a dictionary
# with locations and location counts from the text string
def get_locations_from_text(text):
    locations_dict = {}
    doc = nlp_ner(text)
    for sentence in doc.sentences:
        for token in sentence.tokens:
            if token.ner == 'S-GPE':
                if not token.text in locations_dict.keys():
                    locations_dict[token.text] = 1
                else:
                    locations_dict[token.text] += 1
            else:
                continue
    return locations_dict

You will need to choose which chapter you would like to extract locations from. For this example I will use Chapter 16.

For **Colab** it will look something like this:
`/content/Gibbon_text/gibbon_decline_volume1_chap16.txt`

For **Jupyter** it will look something like this:
`../text/gibbon_decline_and_fall/gibbon_decline_volume1_chap16.txt`

In [7]:
# identify the path to the text file you want to use
path_to_file = "../text/gibbon_decline_and_fall/gibbon_decline_volume1_chap16.txt"
# <-- Insert path to chosen file

In [8]:
# read text from text file
with open(path_to_file, encoding='utf-8', mode='r') as f:
       text  = f.read()

In [9]:
# apply function to get locations and location counts
# this will take a few minutes
locations = get_locations_from_text(text)

In [10]:
# sanity check
locations

{'Jerusalem': 5,
 'Rome': 59,
 'Egypt': 10,
 'Cyprus': 1,
 'Cyrene': 1,
 'Israel': 2,
 'Italy': 23,
 'Tiberias': 1,
 'Syria': 9,
 'Athens': 3,
 'Carthage': 25,
 'Greece': 3,
 'Nazareth': 1,
 'Providence': 1,
 'Palestine': 11,
 'Galilee': 2,
 'Germany': 2,
 'Nerva': 2,
 'Cocaba': 2,
 'Campania': 2,
 'Alexandria': 2,
 'Curubis': 2,
 'Zeugitana': 2,
 'Antioch': 12,
 'Gallienus': 2,
 'Odenathus': 2,
 'Constantius': 2,
 'Tingi': 2,
 'Mauritania': 2,
 'Nicomedia': 14,
 'Lucania': 2,
 'Phrygia': 2,
 'Armenia': 2,
 'Britain': 2,
 'Spain': 4,
 'Maxentius': 4,
 'Tarsus': 2,
 'Cilicia': 2,
 'Illyricum': 4,
 'Tyre': 4,
 'Netherlands': 2}

In [11]:
# you may want to save the locations dictionary
path = '../data' # <-- Path of your choosing
file_name = 'locations_data.json'
with open(file_name, encoding='utf-8', mode='w') as f:
    json.dump(locations, f)

In [12]:
# convert dictionary to dataframe for easier processing
location_count_df = pd.DataFrame.from_dict(locations, orient='index').reset_index().rename(columns={'index':'place_name', 0:'count'})


In [13]:
# preview DataFrame
location_count_df.head()

place_name  count
0  Jerusalem      5
1       Rome     59
2      Egypt     10
3     Cyprus      1
4     Cyrene      1

## Load data from Pleiades

In [14]:
# data from Pleiades, thanks to Peter Nadel!
if not os.path.isfile('places.csv'):  # checkin to see if we have this file or not
    wget.download('https://raw.githubusercontent.com/pnadelofficial/FallDHCourseMaterials/main/places.csv')
if not os.path.isfile('names.csv'):
    wget.download('https://raw.githubusercontent.com/pnadelofficial/FallDHCourseMaterials/main/names.csv')

In [15]:
# load and preview places DataFrame
places_df = pd.read_csv('places.csv')
places_df.head()

created                                        description  \
0  2010-06-24T14:11:06Z  The ancient region known to the Romans as "Gal...   
1  2020-01-10T20:52:00Z  A Roman house in Pompeii (I, 6, 15), also know...   
2  2015-05-28T11:48:45Z  The so-called "House of the Lararium of Achill...   
3  2020-03-20T15:26:53Z  A necropolis with inhumations dating to the fi...   
4  2018-05-30T03:08:22Z  The megalithic defensive circuit of Rusellae d...   

                                             details  \
0   <p>The Barrington Atlas Directory notes: FRA</p>   
1  <p>The house was excavated in 1913 and 1914. T...   
2                                                NaN   
3  <p>A necropolis located close to Monte Bibele ...   
4                                                NaN   

                             provenance                              title  \
0  Barrington Atlas: BAtlas 1 D1 Gallia                             Gallia   
1                              Pleiades                  House of the Ceii   
2                              Pleiades  House of the Lararium of Achilles   
3                              Pleiades         Monte Tamburino necropolis   
4                              Pleiades           Circuit wall of Rusellae   

                                          uri         id  \
0        https://pleiades.stoa.org/places/993        993   
1  https://pleiades.stoa.org/places/999909607  999909607   
2  https://pleiades.stoa.org/places/999909608  999909608   
3  https://pleiades.stoa.org/places/999917206  999917206   
4  https://pleiades.stoa.org/places/999951524  999951524   

   representative_latitude  representative_longitude  \
0                46.705437                  1.013706   
1                40.750010                 14.489506   
2                40.750362                 14.489286   
3                44.272322                 11.375880   
4                42.829089                 11.163588   

                                    bounding_box_wkt  
0  POLYGON ((8.672222 42.4395125, 8.672222 51.981...  
1  POLYGON ((14.4895058 40.7500099, 14.4895058 40...  
2  POLYGON ((14.489286 40.750362, 14.489286 40.75...  
3  POLYGON ((11.3758803 44.2723222, 11.3758803 44...  
4  POLYGON ((11.1635884 42.8290888, 11.1635884 42...

In [16]:
# load and preview names DataFrame
names_df = pd.read_csv('names.csv')
names_df.head()

created description details  \
0  2010-06-24T14:11:06Z         NaN     NaN   
1  2010-06-24T14:11:07Z         NaN     NaN   
2  2015-06-23T17:19:22Z         NaN     NaN   
3  2017-06-26T23:54:15Z         NaN     NaN   
4  2020-12-26T17:59:50Z         NaN     NaN   

                             provenance                       title  \
0  Barrington Atlas: BAtlas 1 D1 Gallia                      Gallia   
1  Barrington Atlas: BAtlas 1 D1 Gallia                       Galli   
2                              Pleiades           House of the Ceii   
3                              Pleiades               Casa dei Ceii   
4                              Pleiades  House of L. Ceius Secundus   

                                                 uri  \
0        https://pleiades.stoa.org/places/993/gallia   
1         https://pleiades.stoa.org/places/993/galli   
2  https://pleiades.stoa.org/places/999909607/nam...   
3  https://pleiades.stoa.org/places/999909607/nam...   
4  https://pleiades.stoa.org/places/999909607/hou...   

                           id   place_id   name_type language_tag  \
0                      gallia        993  geographic          NaN   
1                       galli        993  geographic          NaN   
2  name.2015-06-23.9071239510  999909607  geographic           en   
3  name.2017-06-26.2201864114  999909607  geographic           it   
4   house-of-l-ceius-secundus  999909607  geographic           en   

                attested_form            romanized_form_1 romanized_form_2  \
0                         NaN                      Gallia              NaN   
1                         NaN                       Galli              NaN   
2           House of the Ceii           House of the Ceii              NaN   
3               Casa dei Ceii               Casa dei Ceii              NaN   
4  House of L. Ceius Secundus  House of L. Ceius Secundus              NaN   

  romanized_form_3 association_certainty transcription_accuracy  \
0              NaN               certain               accurate   
1              NaN               certain               accurate   
2              NaN               certain               accurate   
3              NaN               certain               accurate   
4              NaN               certain               accurate   

  transcription_completeness  year_after_which  year_before_which  
0                   complete               NaN                NaN  
1                   complete               NaN                NaN  
2                   complete            1700.0             2100.0  
3                   complete            1700.0             2100.0  
4                   complete               NaN                NaN

In [17]:
# quick example: find 'Roma' in places DataFrame
places_df.loc[places_df['title'] == 'Roma']

created                                    description  \
21483  2018-06-07T19:48:13Z  The capital of the Roman Republic and Empire.   

                                                 details  \
21483  <p>The Barrington Atlas Directory notes: Roma/...   

                                provenance title  \
21483  Barrington Atlas: BAtlas 43 B2 Roma  Roma   

                                           uri      id  \
21483  https://pleiades.stoa.org/places/423025  423025   

       representative_latitude  representative_longitude  \
21483                41.891775                 12.486137   

                                        bounding_box_wkt  
21483  POLYGON ((12.486137 41.891775, 12.486137 41.89...

In [18]:
# quick example: find 'Rome' in names DataFrame
names_df.loc[names_df['romanized_form_1'] == 'Rome']

created                     description details  \
20810  2012-02-04T23:39:48Z  The modern English appellation     NaN   

                                provenance title  \
20810  Barrington Atlas: BAtlas 43 B2 Roma  Rome   

                                                uri    id  place_id  \
20810  https://pleiades.stoa.org/places/423025/rome  rome    423025   

        name_type language_tag attested_form romanized_form_1  \
20810  geographic           en          Rome             Rome   

      romanized_form_2 romanized_form_3 association_certainty  \
20810              NaN              NaN               certain   

      transcription_accuracy transcription_completeness  year_after_which  \
20810               accurate                   complete            1700.0   

       year_before_which  
20810             2100.0

## Extract data from Pleiades data
For each location in we identified from the text, we will extract extract the longitude, latitude, and a description. First we need to find each location in the Pleiades data.

In [19]:
def get_pleiades_id(location):
    """
    Iterates through all of the possible names in the names.csv file
    Returns None if no matched names
    """
    name_row = names_df.loc[names_df['attested_form'] == location]
    if len(name_row) == 1:
        return int(name_row.place_id.iloc[0])
    else:
        name_row = names_df.loc[names_df['romanized_form_1'] == location]
        if len(name_row) == 1:
            return int(name_row.place_id.iloc[0])
        else:
            name_row = names_df.loc[names_df['romanized_form_2'] == location]
            if len(name_row) == 1:
                return int(name_row.place_id.iloc[0])
            else:
                name_row = names_df.loc[names_df['romanized_form_3'] == location]
                if len(name_row) == 1:
                    return int(name_row.place_id.iloc[0])
                else:
                    return None

In [20]:
# apply the above founction to each row in our location count DataFrame and then
# add a new colum with the Pleiades id
location_count_df['pleiades_id'] = location_count_df['place_name'].apply(get_pleiades_id)

In [21]:
# preview new location count DataFrame.
# the NaN means we were unable to find the location in the Pleiades data.
location_count_df.head()

place_name  count  pleiades_id
0  Jerusalem      5          NaN
1       Rome     59     423025.0
2      Egypt     10          NaN
3     Cyprus      1     707498.0
4     Cyrene      1          NaN

In [22]:
# we can drop the rows with NaN values
location_count_df = location_count_df.dropna().reset_index(drop=True)

In [23]:
# preview updated location count DataFrame
location_count_df.head()

place_name  count  pleiades_id
0       Rome     59     423025.0
1     Cyprus      1     707498.0
2     Athens      3     579885.0
3   Carthage     25     314921.0
4   Nazareth      1     678299.0

Now that we have a `pleiades_id` for each location from names.csv, we can use that information to get more data from the places.csv. It would be possible to combine the functions below into one, but I have seperated them out for clarity.

In [24]:
def get_description(pleiades_id):
    """return description from a pleiades id"""
    places_row = places_df.loc[places_df['id'] == pleiades_id]
    if len(places_row) == 1:
        return places_row.description.iloc[0]

In [25]:
def get_uri(pleiades_id):
    """return uri from a pleiades id"""
    places_row = places_df.loc[places_df['id'] == pleiades_id]
    if len(places_row) == 1:
        return places_row.uri.iloc[0]

In [26]:
def get_latitude(pleiades_id):
    """return latitude from a pleiades id"""
    places_row = places_df.loc[places_df['id'] == pleiades_id]
    if len(places_row) == 1:
        return places_row.representative_latitude.iloc[0]

In [27]:
# Challenge: Can you write a function to get the longitude data?
def get_longitude(pleiades_id):
    """return longitude from a pleiades id"""
    places_row = places_df.loc[places_df['id'] == pleiades_id]
    if len(places_row) == 1:
        return places_row.representative_longitude.iloc[0]


In [28]:
# add new column for description
location_count_df['description'] = location_count_df['pleiades_id'].apply(get_description)

In [29]:
# Challenge: can you write the code to add a colmn for the uri?
location_count_df['uri'] = location_count_df['pleiades_id'].apply(get_uri)

In [30]:
# add new column for latitude
location_count_df['latitude'] = location_count_df['pleiades_id'].apply(get_latitude)

In [31]:
# Challenge: can you write the code to add a colmn for the longitude?
location_count_df['longitude'] = location_count_df['pleiades_id'].apply(get_longitude)

Now that we have all the data we need, I am going to make a few little changes to the DataFrame.

In [32]:
# now that we have a uri we don't need the pleiades_id
location_count_df = location_count_df.drop(columns=['pleiades_id'])

In [33]:
# for our purposes we don't really need an index, so I will make the place_name column the index
location_count_df.set_index('place_name', inplace=True)

In [34]:
# final sanity check
location_count_df

count                                        description  \
place_name                                                             
Rome           59      The capital of the Roman Republic and Empire.   
Cyprus          1     Third largest island in the Mediterranean Sea.   
Athens          3  A major Greek city-state and the principal cit...   
Carthage       25  The ancient city of Carthage, located in moder...   
Nazareth        1     An ancient place, cited: BAtlas 69 B4 Nazareth   
Curubis         2      An ancient place, cited: BAtlas 32 G3 Curubis   
Zeugitana       2  Zeugitana is the name given to the northern po...   
Tingi           2  Tingi (modern Tangier) began as a Carthaginian...   
Nicomedia      14  Nicomedia was founded in 712/11 BC as a Megari...   
Lucania         2  An ancient region of south Italy stretching fr...   
Cilicia         2  An ancient region, stretching along the southe...   
Illyricum       4  Illyricum was a Roman province that existed fr...   

                                                uri   latitude  longitude  
place_name                                                                 
Rome        https://pleiades.stoa.org/places/423025  41.891775  12.486137  
Cyprus      https://pleiades.stoa.org/places/707498  35.032266  33.257783  
Athens      https://pleiades.stoa.org/places/579885  37.972634  23.722746  
Carthage    https://pleiades.stoa.org/places/314921  36.850032  10.318000  
Nazareth    https://pleiades.stoa.org/places/678299  32.701029  35.300148  
Curubis     https://pleiades.stoa.org/places/314943  36.575649  10.862461  
Zeugitana   https://pleiades.stoa.org/places/315268  37.250000   9.250000  
Tingi       https://pleiades.stoa.org/places/275736  35.787546  -5.810233  
Nicomedia   https://pleiades.stoa.org/places/511337  40.765191  29.919887  
Lucania     https://pleiades.stoa.org/places/442639  39.885000  17.276944  
Cilicia     https://pleiades.stoa.org/places/658440  36.250000  33.250000  
Illyricum   https://pleiades.stoa.org/places/481865  42.427292  17.965028

## Save location data for further use

In [35]:
# create path and file name variables
path = "../data"# <-- set path variable (not necessary for Colab)
file_name = "gibbon-16-loc.csv"# <-- set file_name variable

In [36]:
# save DataFrame to a .csv file
location_count_df.to_csv(file_name) # <-- For Jupyter you may want to add path

In [37]:
# Colab only
files.download(file_name)

NameError: name 'files' is not defined